# Выбор локации для скважины

Из добывающей компании поступила задача решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой.

Шаги для выбора локации:

- В избранном регионе будем искать месторождения, для каждого определим значения признаков;
- Построим модель и оценим объём запасов;
- Выбирем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль посчитаем равной суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
df1 = pd.read_csv('geo_data_0.csv')
df2 = pd.read_csv('geo_data_1.csv')
df3 = pd.read_csv('geo_data_2.csv')

**Изучим данные первого региона**

In [3]:
df1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
#проверим данные на наличие дубликатов.
df1.duplicated().sum()

0

**Изучим данные второго региона**

In [6]:
df2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
#проверим данные на наличие дубликатов.
df2.duplicated().sum()

0

**Изучим данные третьего региона**

In [9]:
df3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [10]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
#проверим данные на наличие дубликатов.
df3.duplicated().sum()

0

In [12]:
# Проверим дубликаты в столбце 'id' первого датафрейма
print((df1.loc[df1.duplicated(subset=['id'], keep=False)]).sort_values(by='id'))

          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078
97785  bsk9y  0.378429  0.005837   0.160827  160.637302
41724  bxg6G -0.823752  0.546319   3.630479   93

In [13]:
# Проверим дубликаты в столбце 'id' второго датафрейма
print((df2.loc[df2.duplicated(subset=['id'], keep=False)]).sort_values(by='id'))

          id         f0         f1        f2     product
5849   5ltQ6  -3.435401 -12.296043  1.999796   57.085625
84461  5ltQ6  18.213839   2.191999  3.993869  107.813044
1305   LHZR0  11.170835  -1.945066  3.002872   80.859783
41906  LHZR0  -8.989672  -4.286607  2.009139   57.085625
2721   bfPNe  -9.494442  -5.463692  4.006042  110.992147
82178  bfPNe  -6.202799  -4.820045  2.995107   84.038886
47591  wt4Uk  -9.091098  -8.109279 -0.002314    3.179103
82873  wt4Uk  10.259972  -9.376355  4.994297  134.766305


In [14]:
# Проверим дубликаты в столбце 'id' третьего датафрейма
print((df3.loc[df3.duplicated(subset=['id'], keep=False)]).sort_values(by='id'))

          id        f0        f1        f2     product
45404  KUPhW  0.231846 -1.698941  4.990775   11.716299
55967  KUPhW  1.211150  3.176408  5.543540  132.831802
11449  VF7Jo  2.122656 -0.858275  5.746001  181.716817
49564  VF7Jo -0.883115  0.560537  0.723601  136.233420
44378  Vcm5J -1.229484 -2.439204  1.222909  137.968290
95090  Vcm5J  2.587702  1.986875  2.482245   92.327572
28039  xCHr8  1.633027  0.368135 -2.378367    6.120525
43233  xCHr8 -0.847066  2.101796  5.597130  184.388641


Из анализа дубликатов по столбцу **id** видно, что не смотря на дублирующее значение в столбце **id** значения в остальных столбцах - уникальные. Поэтому оставим как есть.

In [15]:
# Удалим столбец id из всех датасетов, т.к. он не несет никакой информации для обучения модели
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)
df3 = df3.drop(['id'], axis=1)

In [16]:
# Выделяем целевые и независимые признаки в регионах
df1_features = df1.drop(['product'], axis=1)
df1_target = df1['product']
df2_features = df2.drop(['product'], axis=1)
df2_target = df2['product']
df3_features = df3.drop(['product'], axis=1)
df3_target = df3['product']

## Обучение и проверка модели

In [18]:
# Делим данные трёх регионов на обучающие и валидационные выборки
df1_features_train, df1_features_valid, df1_target_train, df1_target_valid = train_test_split(df1_features, df1_target, test_size=0.25)
df2_features_train, df2_features_valid, df2_target_train, df2_target_valid = train_test_split(df2_features, df2_target, test_size=0.25)
df3_features_train, df3_features_valid, df3_target_train, df3_target_valid = train_test_split(df3_features, df3_target, test_size=0.25)

In [19]:
# Проверим правильность формирования

print('Тренировочные данные для первого региона', df1_features_train.shape, df1_target_train.shape)
print('Валидационные данные для первого региона', df1_features_valid.shape, df1_target_valid.shape)
print('--------')
print('Тренировочные данные для второго региона', df2_features_train.shape, df2_target_train.shape)
print('Валидационные данные для второго региона', df2_features_valid.shape, df2_target_valid.shape)
print('--------')
print('Тренировочные данные для третьего региона', df3_features_train.shape, df3_target_train.shape)
print('Валидационные данные для третьего региона', df3_features_valid.shape, df3_target_valid.shape)

Тренировочные данные для первого региона (75000, 3) (75000,)
Валидационные данные для первого региона (25000, 3) (25000,)
--------
Тренировочные данные для второго региона (75000, 3) (75000,)
Валидационные данные для второго региона (25000, 3) (25000,)
--------
Тренировочные данные для третьего региона (75000, 3) (75000,)
Валидационные данные для третьего региона (25000, 3) (25000,)


In [20]:
# создаем нашу модель линейной регрессии
model = LinearRegression()

In [21]:
# обучаем модель на данных региона №1
model.fit(df1_features_train, df1_target_train)
df1_predicted = model.predict(df1_features_valid)
mse_df1 = mean_squared_error(df1_target_valid, df1_predicted)
rmse_df1 = mse_df1**0.5
df1_mean = df1_predicted.mean()
print(f'Регион №1. Средний запас сырья: {df1_mean}, RMSE = {rmse_df1}')

Регион №1. Средний запас сырья: 92.49350121856048, RMSE = 37.76594573544278


In [22]:
# обучаем модель на данных региона №2
model.fit(df2_features_train, df2_target_train)
df2_predicted = model.predict(df2_features_valid)
mse_df2 = mean_squared_error(df2_target_valid, df2_predicted)
rmse_df2 = mse_df2**0.5
df2_mean = df2_predicted.mean()
print(f'Регион №2. Средний запас сырья: {df2_mean}, RMSE = {rmse_df2}')

Регион №2. Средний запас сырья: 68.47261919180458, RMSE = 0.8958609139345637


In [25]:
# обучаем модель на данных региона №3
model.fit(df3_features_train, df3_target_train)
df3_predicted = model.predict(df3_features_valid)
mse_df3 = mean_squared_error(df3_target_valid, df3_predicted)
rmse_df3 = mse_df3**0.5
df3_mean = df3_predicted.mean()
print(f'Регион №3. Средний запас  сырья: {df3_mean}, RMSE = {rmse_df3}')

Регион №3. Средний запас  сырья: 94.94420594268205, RMSE = 39.97534330066723


**Вывод:**
Судя по показателю RSME лучше всего себя показала модель 2-го региона, но она сильно уступает значениям среднего запаса сырья по сравнению с другими регионами.


## Подготовка к расчёту прибыли

In [26]:
# Сохраняем ключевые значения для расчетов
regional_budget = 10000000000 # Бюджет на разработку скважин в регионе
one_barrel_income = 450  # Доход с одного барреля сырья
income_per_unit = 450000 # Доход с каждой единицы продукта
wells_count = 200 # Количество лучших скважин для расчета прибыли

In [27]:
# Рассчитываем стоимость разработки одной скважины в регионе
price_one_well= regional_budget/wells_count
print('Стоимость разработки одной скважины в регионе:', price_one_well)

Стоимость разработки одной скважины в регионе: 50000000.0


In [28]:
# Рассчитываем достаточный объём сырья для безубыточной разработки новой скважины
sufficient_volume = price_one_well / income_per_unit
print('Достаточный объём сырья для безубыточной разработки новой скважины:', round(sufficient_volume, 2))

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11


In [29]:
# Сравнение средних запасов предсказанного сырья в регионах по сравнению с объёмом достаточным для безубыточной разработки
means = [df1_mean, df2_mean, df3_mean]

In [31]:
def compare_volume(region_volumes):
    for i in range(len(region_volumes)):
        if region_volumes[i] < sufficient_volume:
            print(f'Средний запас сырья в регионе {i+1} меньше объёма, достаточного для безубыточной разработки на {round(sufficient_volume - region_volumes[i], 2)}')
        else:
            print(f'Средний запас сырья в регионе {i+1} больше объёма, достаточного для безубыточной разработки на {round(region_volumes[i] - sufficient_volume, 2)}')

In [32]:
compare_volume(means)

Средний запас сырья в регионе 1 меньше объёма, достаточного для безубыточной разработки на 18.62
Средний запас сырья в регионе 2 меньше объёма, достаточного для безубыточной разработки на 42.64
Средний запас сырья в регионе 3 меньше объёма, достаточного для безубыточной разработки на 16.17


**Вывод:**


Во всех регионах средний запас сырья меньше объема, достаточного для безубыточной разработки.


Разработка месторождения во втором регионе получается слишком затратным.


Расчитаем прибыль и риски.


## Расчёт прибыли и рисков

In [33]:
# Приведем данные к Series
prediction_df1 = pd.Series(df1_predicted).reset_index(drop=True)
prediction_df2 = pd.Series(df2_predicted).reset_index(drop=True)
prediction_df3 = pd.Series(df3_predicted).reset_index(drop=True)
target_df1 = pd.Series(df1_target_valid).reset_index(drop=True)
target_df2 = pd.Series(df2_target_valid).reset_index(drop=True)
target_df3 = pd.Series(df3_target_valid).reset_index(drop=True)

# возьмем 500 случайных точек
state = np.random.RandomState(12345)
target_subsample_df1 = target_df1.sample(500, replace=True, random_state=state)
predictions_subsample_df1 = prediction_df1[target_subsample_df1.index]
target_subsample_df2 = target_df2.sample(500, replace=True, random_state=state)
predictions_subsample_df2 = prediction_df2[target_subsample_df2.index]
target_subsample_df3 = target_df3.sample(500, replace=True, random_state=state)
predictions_subsample_df3 = prediction_df3[target_subsample_df3.index]

In [34]:
# Функция для расчета прибыли по 200 наилучших из 500 выбранных скважин по предсказанию модели
def revenue(target, probabilities):
    state = np.random.RandomState(12345)
    probs_sorted = probabilities.sort_values(ascending=False).head(wells_count)
    selected = target[probs_sorted.index][:wells_count]
    revenue =  selected.sum() * income_per_unit - regional_budget
    return revenue

In [35]:
print('Прибыль для полученого объема сырья для 1-го региона:',
      round(revenue(target_subsample_df1, predictions_subsample_df1)/1000000,2),'млн. руб.')

Прибыль для полученого объема сырья для 1-го региона: 240.75 млн. руб.


In [36]:
print('Прибыль для полученого объема сырья для 2-го региона:',
      round(revenue(target_subsample_df2, predictions_subsample_df2)/1000000,2),'млн. руб.')

Прибыль для полученого объема сырья для 2-го региона: 141.0 млн. руб.


In [37]:
print('Прибыль для полученого объема сырья для 3-го региона:',
      round(revenue(target_subsample_df3, predictions_subsample_df3)/1000000,2),'млн. руб.')

Прибыль для полученого объема сырья для 3-го региона: 1068.74 млн. руб.


In [38]:
def bootstrap (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        values.append(revenue(target_subsample, preds_subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    hight = values.quantile(0.975)
    loss = values[values<0]
    mean = values.mean()
    print("Средняя прибыль: {:.6f} млрд.руб.".format(mean))
    print("2,5%-квантиль: {:.6f} млрд.руб.".format(lower))
    print("97,5%-квантиль: {:.6f} млрд.руб.".format(hight))
    print("Количество убыточных экспериментов:", loss.count())
    print("Вероятность убытков: {:.2%}".format(loss.count()/len(values)))
    print("Максимальный убыток:", loss.min())
    print("Максимальная прибыль:", values.max())
    print("Количество экспериментов:", len(values))

In [39]:
print('Для 1-го региона')
bootstrap (target_df1, prediction_df1)

Для 1-го региона
Средняя прибыль: 451059347.240609 млрд.руб.
2,5%-квантиль: -53601156.507546 млрд.руб.
97,5%-квантиль: 983571548.648072 млрд.руб.
Количество убыточных экспериментов: 43
Вероятность убытков: 4.30%
Максимальный убыток: -449363914.4861927
Максимальная прибыль: 1323408153.6544876
Количество экспериментов: 1000


In [40]:
print('Для 2-го региона')
bootstrap (target_df2, prediction_df2)

Для 2-го региона
Средняя прибыль: 469111486.936253 млрд.руб.
2,5%-квантиль: 63026441.054432 млрд.руб.
97,5%-квантиль: 890338646.799540 млрд.руб.
Количество убыточных экспериментов: 13
Вероятность убытков: 1.30%
Максимальный убыток: -226478017.35339546
Максимальная прибыль: 1190434181.7969322
Количество экспериментов: 1000


In [41]:
print('Для 3-го региона')
bootstrap (target_df3, prediction_df3)

Для 3-го региона
Средняя прибыль: 380964891.222014 млрд.руб.
2,5%-квантиль: -130295157.217595 млрд.руб.
97,5%-квантиль: 926763397.436772 млрд.руб.
Количество убыточных экспериментов: 81
Вероятность убытков: 8.10%
Максимальный убыток: -529223172.99575233
Максимальная прибыль: 1093189286.5495358
Количество экспериментов: 1000


**Вывод:**
Расчет для 200 лучших точек из 500, выбранных случайным образом, показал, что наибольшую прибыль показал 1-ый и 2-ой регион.

Bootstrap показал, что разработка месторождения во 2-ом регионе наименее рискованная. Максимальная прибыль во всех регионах примерно одинаковая.

**Общий вывод:**


Проведенный анализ показал, что целесообразно разработывать скважины во 2-ом регоине. Это подтверждает низкий показатель RMSE модели, а также низкий процент вероятности убытков (0,4%). Средняя прибыль для данного региона составила 518,68 млн. руб.